In [1]:
import gym
import time
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [9]:
######## Create the envisornment and set up its variables
env = gym.make('FrozenLake-v0')
# env = gym.make('FrozenLake8x8-v0')
action_space_size = env.action_space.n
state_space_size = env.observation_space.n


In [10]:
####### Build the Q table that we'll be using to reference actions
q_table = np.zeros((state_space_size, action_space_size))
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [11]:
num_episodes = 10_000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [12]:
##### Learning Loop
rewards_all_episodes = []

for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 

        # Exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()

        new_state, reward, done, info = env.step(action)

        # Update Q-table for Q(s,a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward 
        
        if done == True: 
            break
    # Exploration rate decay
    exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    rewards_all_episodes.append(rewards_current_episode)

In [13]:
rewards_per_thousand = np.split(np.array(rewards_all_episodes),num_episodes/1000)
print('*'*10,' Rewards Per 1000 Episode ','*'*10,'\n\n')

for i in range(len(rewards_per_thousand)):
    print(f'{(i+1) *1000}: {rewards_per_thousand[i].mean()}')

**********  Rewards Per 1000 Episode  ********** 


1000: 0.051
2000: 0.234
3000: 0.387
4000: 0.573
5000: 0.607
6000: 0.645
7000: 0.697
8000: 0.673
9000: 0.7
10000: 0.696


In [14]:
'''
Possible Actions:
Left  = 0 
Down  = 1
Right = 2
Up    = 3
'''
n = int(np.sqrt(state_space_size))
policy = np.argmax(q_table,axis=1)
moves = ['L','D','R','U']
solution = np.empty(state_space_size,dtype=np.str)
for i,j in enumerate(policy):
    solution[i] = moves[j]

t = ''.join(i for i in solution)
print(f'*** Policy \u03C0 ***\n{t[:n]}\n{t[n:2*n]}\n{t[2*n:3*n]}\n{t[3*n:4*n]}')

*** Policy π ***
LUUU
LLLL
UDLL
LRDL


In [15]:
for e in range(10):
    state = env.reset()
    done = False
    while not done:
        env.render()
        time.sleep(0.09)
        clear_output(wait=True)
        action = np.argmax(q_table[state])
        new_state,reward,done,_ = env.step(action)
        state = new_state
        if reward == 1:
            print('*** Successfull ***')
            time.sleep(1)
            break
        if done:
            print('*** Sorry ***')
            time.sleep(0.3)
            break

*** Successfull ***
